In [ ]:
import os
import torch
import torch.nn.functional as F
import pytorch_lightning as pl;
import importlib
import matplotlib.pyplot as plt
import numpy as np
import data
import utils
import sys
import importlib
import wandb
import copy
import pickle

from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from abc import ABC,abstractmethod
from torch.utils.data import DataLoader, Subset
from copy import deepcopy
from torch.special import logit
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.tensorboard import SummaryWriter
from pytorch_lightning.loggers import WandbLogger
from timeit import default_timer as timer

importlib.reload(data)
importlib.reload(utils)

pl.seed_everything(42)

debug=False

device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
import masking,data
from data import IRAVENDataModule
importlib.reload(masking)
importlib.reload(data)

from SCL_model import SCLTrainingWrapper,SCL

#I-RAVEN dataset



batch_size=8
data=IRAVENDataModule(batch_size=batch_size)
data.prepare_data()
data.setup()
train_dataloader,test_dataloader,val_dataloader=data.train_dataloader(),data.test_dataloader(),data.val_dataloader()


#setup model
scl_kwargs={
    "image_size":160,                            # size of image
    "set_size": 9,                               # number of questions + 1 answer
    "conv_channels": [1, 16, 16, 32, 32, 32],    # convolutional channel progression, 1 for greyscale, 3 for rgb
    "conv_output_dim": 80,                       # model dimension, the output dimension of the vision net
    "attr_heads": 10,                            # number of attribute heads
    "attr_net_hidden_dims": [128],               # attribute scatter transform MLP hidden dimension(s)
    "rel_heads": 80,                             # number of relationship heads
    "rel_net_hidden_dims": [64, 23, 5] 
}
model=SCL(**scl_kwargs)
wrapped_model=SCLTrainingWrapper(model)



kwargs={
    'model':model,
    'train_dataloader':train_dataloader,
    'test_dataloader1':val_dataloader,
    'test_dataloader2':test_dataloader,
    'savedir':'model_ckpts/FFN',
}

X,y,_,_=next(iter(train_dataloader))

masked_scl=masking.MaskedSCLModel(kwargs)
out=masked_scl.forward(X)

In [ ]:
import SCL_model
importlib.reload(SCL_model)


kwargs={
        "image_size":160,                            # size of image
        "set_size": 9,                               # number of questions + 1 answer
        "conv_channels": [1, 16, 16, 32, 32, 32],    # convolutional channel progression, 1 for greyscale, 3 for rgb
        "conv_output_dim": 80,                       # model dimension, the output dimension of the vision net
        "attr_heads": 10,                            # number of attribute heads
        "attr_net_hidden_dims": [128],               # attribute scatter transform MLP hidden dimension(s)
        "rel_heads": 80,                             # number of relationship heads
        "rel_net_hidden_dims": [64, 23, 5] 
    }

model=SCL_model.SCL(**kwargs)
X,y,_,_=next(iter(train_dataloader))

questions,answers=X[:,0:8:,:,:,].unsqueeze(2),X[:,8:,:,:,].unsqueeze(2)
wrapped_model=SCL_model.SCLTrainingWrapper(model)
out=wrapped_model(questions,answers)
